In [51]:
import pandas as pd
import re
from textblob import TextBlob
import spacy as spacy
import numpy as np
nlp = spacy.load("pl_core_news_lg")
from spacy.lang.pl import Polish
parser = Polish()
from spacy.lang.pl.stop_words import STOP_WORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit

In [52]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [53]:
stopwords = list(STOP_WORDS)

# Cleaning and importing data


In [60]:
import string
punctuations=string.punctuation

In [61]:
def tokenizer(sentence):
    mytokens=nlp(sentence)
    mytokens=[word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [62]:
c=tokenizer('W celu pokazu prezentuję następujące zdanie')
c

['cel', 'pokaz', 'prezentować', 'następujący', 'zdanie']

In [64]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}


def clean_text(text):     
    return text.strip().lower()

In [66]:
vectorizer=CountVectorizer(tokenizer=tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

In [67]:
df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')
df

<ipython-input-67-4df8a0abd9e6>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')


,Text,label,label;;;;;;
0,"Taksowak przyjezdza na czas , wydaje sie , ze ...",label,z_zero;;;;;;
1,Do tej pani trafili śmy z naszą 2 - miesięczna...,label,z_zero;;;;;;
2,"Wyróżnikiem , a także ciekawym elementem całeg...",label,z_zero;;;;;;
3,"W zależności od pory dnia , w naszych organizm...",label,z_zero;;;;;;
4,"Polecam wszystkim , zwłaszcza tym , dla któryc...",label,z_plus_m;;;;;;
...,...,...,...
45959,Na plus zaliczył by m też wbudowane gniazda US...,label,z_plus_m;;;;;;
45960,Bardzo blisko ( 3 - 5 min . drogi ) do centrum...,label,z_plus_m;;;;;;
45961,"Fachowo , prawie bezboleśnie i skutecznie , bo...",label,z_plus_m;;;;;;
45962,Podsumowując : 1 . umówił em się na prywatną w...,label,z_minus_m;;;;;;


In [68]:
def getRidOff(text):
    text=re.sub(r';','',text)
    text=re.sub(r'"','',text)
    return text
df['label;;;;;;']=df['label;;;;;;'].apply(getRidOff)

In [69]:
def cleanText(text):
    text=re.sub(r':','',text)
    text=re.sub(r';','',text)
    text=re.sub(r'[0-9]+','',text)
    text=re.sub(r'  ',' ',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'-','',text)
    text=re.sub(r',','',text)
    text=re.sub(r'"','',text)
    return text
df['Text']=df['Text'].apply(cleanText)

In [70]:
df.drop('label',axis=1,inplace=True)

In [71]:
df=df.rename(columns={'label;;;;;;':'label'})
df.to_csv('daneImport.csv')

In [73]:
df =df[df.label != 'z_zero']
df =df[df.label != 'z_amb']
df

,Text,label
4,Polecam wszystkim zwłaszcza tym dla których ...,z_plus_m
6,Wszystko stare i okropne .,z_minus_m
7,Arogancki właściciel ustawiający gości .,z_minus_m
10,Pokój relaksu z Jacuzzi różnorodne usługi ma...,z_plus_m
11,Hotel na bardzo dobrym poziomie zarówno jeśli ...,z_plus_m
...,...,...
45958,W ostatni dzień kucharz podał nam surowego klo...,z_minus_m
45959,Na plus zaliczył by m też wbudowane gniazda US...,z_plus_m
45960,Bardzo blisko ( min . drogi ) do centrum i ma...,z_plus_m
45961,Fachowo prawie bezboleśnie i skutecznie bo e...,z_plus_m


# Splitting data and training model


In [81]:
X=df['Text']

In [82]:
y=df['label']


In [83]:
y

4         z_plus_m
6        z_minus_m
7        z_minus_m
10        z_plus_m
11        z_plus_m
           ...    
45958    z_minus_m
45959     z_plus_m
45960     z_plus_m
45961     z_plus_m
45962    z_minus_m
Name: label, Length: 29200, dtype: object

In [84]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [85]:
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [86]:
X_train

28460            Pokoje bardzo czyste  ładnie urządzone . 
12234    Bardzo polecam wszystkim kobietom kontakt z ty...
25443    Dodam  że mama była ciężko chora na nowotwór i...
1068     Zarowno lek jak i srodki do pielegnacji skory ...
26237      Obsługa miła  wszyscy uśmiechnięci i pomocni . 
                               ...                        
33928    Na stołówce notorycznie czegoś brakowało taler...
8442     Moim zdaniem nadaje się do interwencji Izby Le...
1389     Wystraszym mnie nieziemsko  zakazal kontynuacj...
24885                 Ogólne bardzo słabo i nie polecam . 
37200    Zadał tylko ogólnikowe pytania typu gdzie mies...
Name: Text, Length: 23360, dtype: object

In [87]:
Y_train

28460     z_plus_m
12234     z_plus_m
25443    z_minus_m
1068     z_minus_m
26237     z_plus_m
           ...    
33928    z_minus_m
8442     z_minus_m
1389     z_minus_m
24885    z_minus_m
37200    z_minus_m
Name: label, Length: 23360, dtype: object

In [88]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000024B2054D670>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function tokenizer at 0x0000024B2065C3A0>)),
                ('classifier', LinearSVC())])

In [89]:
samplePredict=pipe.predict(X_test)

In [90]:
for (sample,pred) in zip(X_test,samplePredict):
    print(sample,"Prediction=>",pred)

Wyposażenie pozostawia wiele do życzenia ( stare  zużyte ) .  Prediction=> z_minus_m
Płacąc grubo ponad zł za wizytę u lekarza  spodziewał em się kompetencji  wiedzy i pomocy .  Prediction=> z_minus_m
Cytuję „ Pacjent zgłosił się z żądaniem zlecenia badań do sanatorium w ramach NFZ  odmówił em zlecenia badań na życzenie NFZ  zlecił em badania zgodnie z dokumentacją pacjenta w Lux Med . . ( tu następuje wymienienie chorób ) …  Prediction=> z_minus_m
Obsługa przeprosiła za niedogodności  ale niestety nie zatarło to złego wrażenia .  Prediction=> z_minus_m
Brak poczucia obecności ochrony i bezpieczeństwa .  Prediction=> z_plus_m
Natomiast śniadanie które było w cenie noclegu było bardzo zróżnicowane i smaczne .  Prediction=> z_plus_m
Wizyta u Pani Doktor nie należała do najprzyjemniejszych .  Prediction=> z_minus_m
zażartował sobie  że mąż nie jest potrzebny do zajścia w ciąży .  Prediction=> z_plus_m
Bardzo dokładna  żaden ginekolog wcześniej nie zbadał mnie tak dokładnie .  Prediction=>

Odradzam serdecznie wszystkim .  Prediction=> z_minus_m
Polecam ! ! !  Prediction=> z_plus_m
Jeden jedyny plus  blisko morza reszta same minusyWłaściciele bardzo nie mili nieprzyjemni dla wczasowiczów wręcz pyskaci Zakaz wejścia komukolwiek na teren pensjonatu pilnuja jak psy nawet jak przyjedzie ktos odwiedzić kogoś z rodziny Jedzenie tragedia lepiej w szpitalu dająPokoje żadna rewelacja brak lodówki .  Prediction=> z_plus_m
własciwie nic się nie dowidział em  podczas badania usłyszał em   wszystko mięciutko  i że jest cos zaczerwienione  ale co to jest to niewiadomo  recepta na czopki i jakby się cos działo to   proszę przyjsć   akurat za zł na wizytę która razem z badaniem trwała tyle mniej więcej ile czekałem pod gabinetem  pani dr . przy mnie zaczęła pisać podsumowanie  dając mi do zrozumienia że to koniec wizyty  spodziewał em się większego zaangażowania lekarza tym bardziej że jestem po radioterapi  szału nie ma ! ! !  Prediction=> z_minus_m
Gdy wyraził em swoje wątpliwości i ch

Zdezorientowany i cierpiący ( szukamy przecież pomocy ) pacjent wychodzi i nie wie co dalej .  Prediction=> z_plus_m
Hotel De Goezeput był idealny dla naszych potrzeb podczas wizyty w Brugii i po przyjechaniu po raz pierwszy z USA .  Prediction=> z_plus_m
Dla mnie trochę wyrzucone pieniądze porównał by m ją do tarczy Nortona MDC  Norton cenowo dużo tańsza a efekty strzępienia identyczne  do tego Norton moim zdaniem przez większość czasu życia tarczy tnie i ukosuje na tym samym poziomie nie widać aż tak bardzo sukcesywnego zużywania tarczy .  Prediction=> z_minus_m
Nie wiem czy to ze względów oszczędnościowych czy z braku umiejętności kucharza ale dno totalne . . .  Prediction=> z_minus_m
Brakowało też światła słonecznego .  Prediction=> z_minus_m
Ustawienie harmonogramu dziecinnie proste .  Prediction=> z_minus_m
Ponownie dzwonię  podobno prosili o spokój .  Prediction=> z_minus_m
Niestety  obsługa hotelu nie była zbyt chętna do wyegzekwowania zapisanej w regulaminie hotelu ciszy nocne

Sposób w jaki rozmawia z pacjentem jest nieżyczliwy i lekceważący .  Prediction=> z_minus_m
Jeżeli kończyło sie cos na kolacje nie było donoszone  szybko zbierane talerze nawet z nieskończonym jedzeniem  szybko podawana kolacja z naciskiem zeby szybko skończyć i zabierać jedzenie tak samo przy śniadaniach .  Prediction=> z_minus_m
Facet jest spoko  można naprawde pograc i nie wtrąca sie do gry .  Prediction=> z_plus_m
Jeżeli chodzi o leżaki mąż codziennie rano kładł ręczniki na plażowych tak powiedzmy     nigdy nie zabrakło z rana pustych  nie zaobserwowała m też ludzi leżących na piasku  więc myślę że to kwestia organizacji .  Prediction=> z_minus_m
A na koniec stracił do nas cierpliwość i mimo wcześniejszych ustaleń dziecko wymagające jeszcze okresowo respiratora ( bez uprzedzenia nas ) przekazał do innego szpitala . . .  Prediction=> z_plus_m
Doskonale słucha  zadaje ciekawe pytanie jak również potrafi trafnie ocenić sytuację i opisać problem .  Prediction=> z_plus_m
Natomiast troch

Jak chcialam normalnego drika musialam placic .  Prediction=> z_minus_m
Swoje lata świetności ma już dawno za sobą .  Prediction=> z_minus_m
Do hotelu trafili śmy zachęceni niską ceną i lokalizacją w Pafos .  Prediction=> z_plus_m
Zostali śmy jedynie poinformowani  że jedna prywatna wizyta to za mało na zaświadczenie .  Prediction=> z_minus_m
WOW Dzięki makrą można na prawdę zrobić wszystko  zbindować każdy przycisk pod siebie lub pod dany program .  Prediction=> z_plus_m
Po drugim kolokwium stwierdziła  że mam błąd rachunkowy w pierwszym zadaniu i ona dalej nie będzie sprawdzała  dała mi za wszystko punktów !  Prediction=> z_minus_m
Śniadania smaczne  miła obsługa zarówno recepcji jak i baru .  Prediction=> z_plus_m
A ludzie chcą w to grać i grają  na na trzy  ale hotelu na taśmy nie stać ! ! ! !  Prediction=> z_minus_m
Mimo starań obsługi  niektóre stoliki przez długi czas pozostają nieposprzątane po poprzednich gościach .  Prediction=> z_minus_m
Próbowała m skorzystać  ale nie za wi

Dobrze  że to była tylko jedna noc w podróży służbowej . . .  Prediction=> z_minus_m
Maz stwierdzil  ze to najgorszy placek po wegiersku jaki jadl a zjadl juz sporo  bo to ulubione jego danie  ktore zawsze podawane jest z surowka .  Prediction=> z_minus_m
Mialam wrazenie  ze dr mnie troche nawet probowal zniechecic  co mi sie nie spodobalo .  Prediction=> z_minus_m
Środek czyszczący z powierzchni czyszczonej bardzo szybko samoistnie odparowuje .  Prediction=> z_minus_m
Czytali śmy wcześniej opinie o tym  że w hotelach gorzej traktują Polaków oraz że Itaka ma najgorsze pokoje  ale po raz pierwszy na własnej skórze to odczuli śmy .  Prediction=> z_minus_m
Wybierajac droge przez park mamy piekny spacer i w ciągu min jestesmy w samym centrum miasta .  Prediction=> z_plus_m
Strojenie kanałów to pomyłka wszystko działa powolnie  język menu nie do końca jest całkowicie polski .  Prediction=> z_minus_m
Nie poinformował mnie o mozliwościach przeciwdziałania skutkom pooperacyjnym  kłamał na spot

In [91]:
print("Accuracy: ",pipe.score(X_test,Y_test))
print("Accuracy: ",pipe.score(X_test,samplePredict))

Accuracy:  0.8102739726027397
Accuracy:  1.0


In [92]:
print(classification_report(Y_test,samplePredict))

              precision    recall  f1-score   support

   z_minus_m       0.85      0.83      0.84      3430
    z_plus_m       0.76      0.79      0.77      2410

    accuracy                           0.81      5840
   macro avg       0.80      0.81      0.81      5840
weighted avg       0.81      0.81      0.81      5840

